# EEG analysis

In [ ]:
import os
import time
import mne
import numpy as np
import pandas as pd

import bsl
from bsl import StreamPlayer, datasets
from bsl.externals import pylsl  # distributed version of pylsl
from bsl.triggers import TriggerDef

import liesl
import pylsl

import math
import matplotlib
import matplotlib.pyplot as plt

# %matplotlib qt
# mne.set_config('MNE_BROWSER_BACKEND', 'qt')

In [ ]:
from idun_guardian_client.igeb_api import GuardianAPI
import datetime

api = GuardianAPI()
device_id = ""

# get a list of all recordings
recording_list = api.get_recordings_info_all(
    device_id=device_id
)  # Device ID is derived from the MAC address of the earbud and in the log file

recordings = sorted(recording_list, key=lambda x: datetime.datetime.fromisoformat(x['startDeviceTimestamp'][:-5]))
recording_id = recordings[-1]["recordingID"]
# download recording
api.download_recording_by_id(
    device_id=device_id, recording_id=recording_id
)
print(recordings[-1])

path = os.path.join("recordings", recording_id, f"{recording_id}_eeg.csv")
df = pd.read_csv(path)
info = mne.create_info(sfreq=250, ch_names=['ch1'], ch_types=['eeg'])
data = np.array([df['ch1'].values])
raw = mne.io.RawArray(data=data, info=info)
raw.filter(1,30)

## Convert it into MNE raw format

In [ ]:
raw.plot(scalings='auto')

In [ ]:
raw.plot_psd(fmin=0, fmax=40, dB=True, estimate='power')